In [71]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
from sortedcontainers import SortedDict
import matplotlib.pyplot as plt
import seaborn as sns
from  matplotlib.ticker import PercentFormatter
import utils.graph_utils as graph_utils
from utils.IO_utils import load_ground_true
from sklearn import metrics

In [72]:
# settings
name = "Dblp"
come_model_type = "BGMM"
ks = [2, 5, 10, 20]

In [73]:
# import graph and true labels

# graph
G = graph_utils.load_matfile(f"../../data/{name}/{name}.mat", undirected=True)

# labels_true
labels_true, _ = load_ground_true(path=f"../../data/{name}", file_name=name)

In [ ]:
# import node embeddings and predicted labels
dfs = {}
for k in ks:
    # import node embeddings
    df = pd.read_csv(
        f"./data/{name}_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-219_ind-219_k-{k}_ds-0.0_type-{come_model_type}.txt",
        sep="\t| ",
        header=None
    )
    df = df.rename(columns={0: 'node'})
    df.set_index(['node'], inplace=True)

    # import predicted labels
    labels = pd.read_csv(f"./data/labels_pred_{come_model_type}_{k}.txt", header=None)
    labels = labels.rename(columns={0: 'label'})
    labels.label = labels.astype(int)

    # join labels to embeddings
    df = df.join(labels)

    display(df)
    dfs[k] = df

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


# Community Detection

In [ ]:
# NMI

for k in dfs:
    print("K =", k)
    df = dfs[k]
    labels_pred = df.label.to_numpy()

    nmi = metrics.normalized_mutual_info_score(labels_true, labels_pred)
    print("NMI: ", nmi)

In [ ]:
# Conductance

for k in dfs:
    print("K =", k)
    df = dfs[k]
    conductance_min = 1
    for i in range(k):
        nodes = df[df.label == i].index.to_numpy()
        conductance_i = nx.conductance(G, nodes)
        print(f"  i{i} conductance: {conductance_i}")
        conductance_min = min(conductance_min, conductance_i)
    print(f"=>conductance: {conductance_min}")

# Node Classification

In [ ]:
# train LibSVM classifier on 90% of data and test on 10%.

#TODO

In [ ]:
def emb_from_row(row):
    """get node embedding np array from df row"""
    return row.drop('label').to_numpy().flatten()
